In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import json
f = open('/content/drive/My Drive/cns/data.json',)
data = json.load(f)
f.close()

In [0]:
mails = pd.read_csv("/content/drive/My Drive/spam_ham_dataset.csv", header=0)
mails.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [0]:
data_train, data_test, label_train, label_test = train_test_split(mails['text'], mails['label'],test_size=.10)

In [0]:
tagged_mails = []
for mail_text, label in zip(data_train, label_train):
  tagged_mail = TaggedDocument(words=word_tokenize(mail_text.lower()), tags=[label])
  tagged_mails.append(tagged_mail) 

In [0]:
stop_words = set(stopwords.words('english')) 

In [0]:
max_epochs = 50
vec_size = 100
alpha_val = 0.025
min_word_count = 1

In [0]:
model = Doc2Vec(vector_size=vec_size, alpha=alpha_val, 
                min_count=min_word_count)
model.build_vocab(tagged_mails)
model.train(tagged_mails, total_examples=model.corpus_count,
             epochs=max_epochs)

In [0]:
model.save("/content/drive/My Drive/cns/d2v.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
mail_vectors = []
count_list_train = []
for text, label in zip(data_train, label_train):
  mail_text = word_tokenize(text.lower())
  count = 0
  for word in mail_text:
    if word in stop_words:
      continue
    if not word.isalpha():
      continue
    if len(word) <= 1:
      continue
    if word in data:
      count=count+1
  count_list_train.append(count)
  mail_vector = model.infer_vector(mail_text)
  mail_vectors.append(mail_vector)

In [0]:
test_vectors = []
count_list_test = []
for text, label in zip(data_test, label_test):
  test_text = word_tokenize(text.lower())
  count = 0
  for word in test_text:
    if word in stop_words:
      continue
    if not word.isalpha():
      continue
    if len(word) <= 1:
      continue
    if word in data:
      count=count+1
  count_list_test.append(count)
  test_vector = model.infer_vector(test_text)
  test_vectors.append(test_vector)

In [0]:
df = pd.DataFrame(data=mail_vectors)
df['spamcount']=count_list_train
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,spamcount
0,2.063180,0.814399,2.957657,-0.059860,-1.683448,-1.390265,2.129796,-0.309502,0.900936,1.226010,-2.687712,0.128964,-0.362772,-2.839307,3.033495,-0.227723,0.126334,1.946337,-0.063552,1.670228,-1.752564,-0.219945,-0.632178,-0.541157,-0.851565,-0.127478,0.452179,-2.044406,0.420111,0.181945,2.129520,2.631207,-1.873802,-2.327163,1.150350,-0.531122,-0.419876,1.478702,0.456691,-1.088295,...,1.045360,0.179422,-0.599545,-0.068133,2.700160,1.288011,-0.037043,1.557546,-0.917605,-1.429407,-1.669331,-2.430813,2.027117,0.942375,-1.448617,0.520152,-0.654034,0.394732,-2.475667,-0.313339,0.335338,1.132087,2.476835,1.564100,1.685876,-1.235178,-0.542109,1.928463,-0.726473,-1.135777,2.705113,1.735896,3.093116,2.005049,0.754860,2.190536,1.073539,1.341245,0.241716,4
1,1.247429,0.890817,2.988337,0.544875,-0.478917,-1.557250,2.787874,1.056468,0.957269,2.882435,-2.797014,0.209970,-0.784591,-2.281419,2.388414,-1.032207,1.002719,3.070910,-1.157084,0.647537,-2.175522,-0.023072,-0.057396,0.949909,-0.812420,0.619261,0.475333,-0.536915,0.025035,0.211131,1.622604,1.750070,-1.828462,-1.258944,0.235456,-0.340990,0.047214,0.236314,0.693204,-1.001657,...,0.743539,0.217700,-0.366706,-1.036036,4.323206,1.301122,-0.290657,1.185734,-1.701751,-1.547175,-0.307930,-3.373655,1.048477,1.812609,-1.117297,0.466282,0.288455,-0.428043,-2.045638,-0.219388,1.161336,1.395826,3.323405,1.644944,0.229891,-1.565555,-1.720623,1.434580,-0.226013,-0.597744,4.257090,-0.158030,3.291736,1.738481,0.983736,2.473027,0.877872,0.557662,0.144976,17
2,0.888197,0.674511,0.467519,-0.707809,-0.237797,0.405560,0.286953,0.369175,0.881110,2.426437,-0.628534,0.951049,-0.427744,-1.468407,2.703897,-0.318783,1.014846,2.878367,0.320132,0.817281,-0.876759,-0.189257,-1.444919,1.024053,0.857689,1.142788,0.603006,-1.506937,-0.070646,-0.537228,1.788249,1.418779,-1.550779,-0.953503,0.305420,0.481120,1.389254,0.454712,0.814917,-0.490871,...,-0.532596,0.100451,-0.319250,-0.162877,1.695886,-0.959987,0.412400,0.290210,-2.080968,-1.231540,-1.055914,-1.648834,0.378367,0.723123,0.487455,0.620622,-0.398344,-0.199922,-1.899530,0.798732,0.589257,-0.277695,4.010310,0.863458,0.524781,0.169933,-1.658385,0.648463,-0.121579,1.211807,2.337729,1.206703,2.608288,0.920717,2.011916,2.302549,3.484094,0.175866,0.501303,6
3,0.781486,0.108593,0.510558,-0.250853,-0.665434,0.273604,0.362987,1.377375,0.531226,2.015775,-1.061931,0.575864,0.033518,-0.806925,1.047438,-1.059810,0.359778,1.534228,0.178698,0.050501,-0.128366,0.179952,-1.082258,0.069454,0.069781,1.027232,0.518894,-1.523881,0.298953,0.289862,0.586244,0.072831,-0.501880,-0.891076,0.463198,0.018157,0.122447,0.275765,-0.198402,-1.131646,...,-0.296150,0.477864,-1.051381,0.081017,1.276221,0.055104,0.350189,-0.087998,-0.192077,-0.776828,-0.647720,-1.541486,1.260619,0.523965,-0.999136,0.508917,-0.834355,0.010463,-1.747217,0.285009,1.277198,0.071423,1.728858,0.909015,0.898353,-0.660921,-0.516506,0.706463,-0.279874,1.342952,0.978231,1.316935,0.967698,0.940673,0.280221,1.555492,0.956216,0.667861,-0.217793,3
4,1.601823,0.083276,1.208442,-0.049826,-0.374542,-0.629493,0.172318,0.616353,0.071857,2.001655,-1.328803,0.562365,0.067720,-0.951336,0.843842,-1.221161,0.963679,1.510965,-0.018835,0.166406,-0.465219,0.706497,-0.784645,0.883503,0.023846,0.160865,0.685005,-0.760768,0.273806,0.004572,1.310859,0.693907,-1.129739,-0.000289,-0.201361,-0.055754,0.694619,0.150064,-0.193470,-0.620011,...,0.296081,-0.120447,-0.495082,0.630748,1.196474,-0.140308,-0.091313,-0.185871,-1.117975,-0.816749,-0.937510,-1.175285,0.367016,0.738211,-0.326152,0.198089,0.288672,0.253140,-0.901164,0.218493,0.324604,0.501709,1.532137,0.426077,0.884226,-0.452878,-1.239958,-0.087841,-0.904655,0.581341,1.415494,0.696710,0.189033,1.021361,0.656420,1.506163,0.737520,-0.254558,-0.339708,2


In [0]:
df_test=pd.DataFrame(data=test_vectors)
df_test['spamcount']=count_list_test
df_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,spamcount
0,0.244401,3.780029,-1.060506,-0.159457,-0.488590,-1.822124,3.546292,-0.309780,-1.223114,1.705852,-4.208495,-1.458492,0.382847,-1.099144,5.915428,1.885826,0.216350,2.313834,0.001858,2.177803,-2.438396,3.672366,1.336149,0.092535,0.432251,-2.651062,1.473642,-5.688504,-0.468530,-0.353929,-0.872560,1.809937,-2.496255,-0.385261,2.585231,2.080072,1.247723,0.642915,0.856534,-1.430649,...,1.775982,-1.274006,-2.193284,-0.262680,6.996186,0.771003,0.681922,3.766701,-3.043590,0.430038,0.153636,-7.093183,1.802384,-0.509373,-0.897775,0.965196,-1.933750,1.038503,-2.694595,-0.852100,0.642042,2.525986,2.432692,2.486484,5.820993,-2.741726,-1.087582,-1.560713,-3.926286,-0.039118,6.137163,0.875099,5.770002,4.398241,-0.605930,3.034302,2.021688,3.494969,1.504424,44
1,1.575464,0.336648,1.428485,0.109249,-1.400513,0.639814,-0.771730,3.061959,-0.289604,0.991432,-0.977163,0.470413,-1.417495,-0.917432,-0.214296,-0.407046,1.400821,2.999294,0.084024,2.121950,-2.264429,-0.188337,-0.069641,-0.351989,1.205064,-0.400332,0.787784,-1.099921,-0.679030,-0.660007,0.425174,0.018369,0.138969,-1.279748,0.844986,-0.593166,0.062445,-0.005643,0.557107,-1.481393,...,0.162222,-1.687375,-0.896741,0.238168,1.964834,-0.375221,-0.491097,0.205287,-1.062247,-1.384976,-1.732449,-1.384204,0.700334,1.031844,-0.584535,0.385446,1.020583,-0.155129,0.809200,-0.011757,2.085131,-0.204332,1.916819,0.966272,0.379873,1.162199,-0.739896,0.480670,-1.633334,0.348719,1.987824,1.296718,2.306034,1.634913,2.579198,1.749489,1.201888,1.037125,0.565125,3
2,0.139682,0.545045,0.318559,-0.285954,-0.049687,-0.038679,0.577129,0.250073,-0.158525,0.580048,-0.306761,-0.068993,-0.072521,-0.084770,0.311180,-0.437765,-0.075946,0.452460,-0.063126,0.123046,-0.228126,0.032337,-0.102951,0.097082,0.038952,0.251616,-0.018778,-0.390548,0.119503,-0.223295,0.188375,-0.062489,-0.206403,-0.206909,0.318654,-0.174199,0.153232,-0.067777,0.100945,-0.018025,...,0.255081,0.310508,-0.236702,-0.023748,0.277275,0.138666,-0.337798,0.299208,-0.147279,-0.114312,-0.276407,-0.449589,0.375847,0.348505,-0.234384,0.163485,-0.345952,-0.236704,-0.197653,0.034090,0.080300,0.029046,0.362189,-0.105524,0.209593,-0.063984,-0.093129,0.401526,-0.165294,0.166042,0.423213,0.011831,0.301912,0.155715,0.180108,0.560152,0.053696,-0.084605,-0.258435,1
3,0.727737,0.266512,3.927508,-0.007201,-1.733994,1.599379,3.017414,0.124803,0.798537,4.439755,-3.056595,1.926590,-0.953367,-0.899965,2.215559,-1.543651,-0.667570,2.776030,-1.332443,-0.099933,-2.126801,1.593484,-0.548350,0.213622,0.767148,1.112745,-1.117006,-0.570105,-0.905657,0.296386,3.816000,0.751400,0.706674,-0.781202,0.604422,-3.772844,1.733816,-0.255198,0.568314,-1.801492,...,-2.263100,-0.479562,-0.056791,-0.344183,1.973022,-1.856318,0.901286,-0.880696,-1.375705,-4.713405,-0.425148,-3.208520,1.911098,0.331582,-0.422985,1.159014,0.531821,-2.643800,-3.730640,0.103521,0.383324,1.856357,5.253921,2.196033,1.579790,-1.145417,-5.926458,2.267905,-1.437368,0.700096,4.099156,2.360307,5.032474,3.503649,5.204229,6.657978,3.595673,0.361933,-0.233039,24
4,1.457495,1.417232,1.500198,0.608155,0.399805,0.393475,1.373795,1.255143,0.560537,1.230263,-2.300946,0.733550,-1.034870,-0.852779,1.224370,-1.676055,0.440998,1.475052,-0.076842,0.357225,-0.283616,-0.655688,-0.486941,-0.170794,1.102354,0.226529,0.497193,-1.544128,-0.124655,0.380814,1.228967,0.867821,-0.882223,-0.769478,0.678540,-0.941289,0.459944,-0.411797,1.178867,-1.777238,...,-0.879865,0.229010,-1.396401,-1.308155,2.307144,-1.382360,-0.371145,-0.194674,-0.962334,-0.336597,-1.712055,-2.428956,1.264139,1.141794,-0.478963,0.264820,0.731122,-0.643279,-0.762760,-0.160006,0.340610,-0.101448,1.366451,1.567876,-0.002487,-0.510654,-1.699781,-0.216148,-0.403471,0.382961,1.928186,0.538187,1.497088,0.753293,0.671870,2.164219,1.711915,0.2

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [0]:
rf = RandomForestClassifier(max_depth=10, random_state=0)
rf.fit(df,label_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
lr = LogisticRegression(random_state=0)
lr.fit(df,label_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
sv = SVC()
sv.fit(df,label_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
dt = DecisionTreeClassifier(max_depth=10)
dt.fit(df,label_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [0]:
nb = GaussianNB()
nb.fit(df,label_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 5), random_state=1)
mlp.fit(df,label_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 5), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
test_predict_rf = rf.predict(df_test)
accuracy_score(test_predict_rf,label_test)

0.8957528957528957

In [0]:
test_predict_lr = lr.predict(df_test)
accuracy_score(test_predict_lr,label_test)

0.944015444015444

In [0]:
test_predict_sv = sv.predict(df_test)
accuracy_score(test_predict_sv,label_test)

0.9305019305019305

In [0]:
test_predict_dt = dt.predict(df_test)
accuracy_score(test_predict_dt,label_test)

0.7741312741312741

In [0]:
test_predict_nb = nb.predict(df_test)
accuracy_score(test_predict_nb,label_test)

0.7297297297297297

In [0]:
test_predict_mlp = mlp.predict(df_test)
accuracy_score(test_predict_mlp,label_test)

0.9401544401544402

In [0]:
correctly_classified = 0
incorrect_spam = 0
incorrect_ham = 0
for text, label in zip(data_test, label_test):
  mail_text = word_tokenize(text.lower())
  mail_vector = model.infer_vector(mail_text)
  predicted_label = model.docvecs.most_similar([mail_vector], topn = 1)
  predicted_label = predicted_label[0][0]
  if predicted_label == label:
    correctly_classified += 1
  elif label == "spam":
    incorrect_spam += 1
  else:
    incorrect_ham += 1


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
print("Correct = " + str(correctly_classified))
print("Wrong Spam = " + str(incorrect_spam))
print("Wrong Ham = " + str(incorrect_ham))

Correct = 427
Wrong Spam = 89
Wrong Ham = 2
